In [1]:
import pickle
import numpy as np
from collections import defaultdict
def ParseDataFromFile(f):
    for l in open(f):
        yield eval(l)
data=list(ParseDataFromFile("train.json"))
data[0]

{'businessID': 'B408037852',
 'categories': [u"Women's Clothing Store",
  u'Fashion Accessories Store',
  u'Shoe Store'],
 'rating': 5.0,
 'reviewHash': 'R471510664',
 'reviewText': u"This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.",
 'reviewTime': u'Mar 24, 2013',
 'unixReviewTime': 1364143460,
 'userID': 'U093387342'}

In [2]:
dusertimes={} #18793 users
mapU2number=[]
j=0
for i in data:
    if i['userID'] not in dusertimes:
        mapU2number.append(i['userID'])
        dusertimes[i['userID']]=1
        j+=1
    else:
        dusertimes[i['userID']]+=1
dU2number={}
for k,v in dusertimes.iteritems():
    try:
        dU2number[mapU2number.index(k)]=v
    except:
        print k


In [3]:
#21321 business
dbusinesstimes={} #21321 business 'B758492670': 8, 'B216830637': 4, 'B109727892': 7,
mapB2number=[]#'B408037852','B410583770','B423621081','B088348224',
j=0
for i in data:
    if i['businessID'] not in dbusinesstimes:
        mapB2number.append(i['businessID'])
        dbusinesstimes[i['businessID']]=1
        j+=1
    else:
        dbusinesstimes[i['businessID']]+=1
dB2number={}#0: 6, 1: 21, 2: 67, 3: 32, 4: 89,
for k,v in dbusinesstimes.iteritems():
    try:
        dB2number[mapB2number.index(k)]=v
    except:
        print k

In [7]:
dUB = defaultdict(set)
for l in data:
    user,business = mapU2number.index(l['userID']),mapB2number.index(l['businessID'])
    dUB[user].add(business)

In [30]:
def knn(d,k):
    rows=len(d)
    ans=[[0]*k for i in range(len(d))]
    for i in range(rows):
        recordcommon=[0]*rows
        for j in range(rows):
            distance=len(d[i]&d[j])
            recordcommon[j]=distance
        for m in range(k):
            maxtmp=max(recordcommon)
            ans[i][m]=recordcommon.index(maxtmp)
            recordcommon[recordcommon.index(maxtmp)]=0
    return ans

In [31]:
ans=knnU=knn(dUB,5)

In [ ]:
X=[]
Y=[]
for user in range(1000):#len(dUB)): #18793
    friend={}
    w1=len(dUB[user])/10.642260416112382
    for neighor in ans[user]:
        for item in dUB[neighor]:
            if item not in friend:
                friend[item]=1.0/len(dUB[neighor])
            else:
                friend[item]+=1.0/len(dUB[neighor])
    for i in friend:
        X.append([1,friend[i]/w1])
        Y.append(1 if i in dUB[user] else 0)